In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [4]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
le_gender=LabelEncoder()
df["Gender"]=le_gender.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
ohe=OneHotEncoder()
geo_encoder=ohe.fit_transform(df[["Geography"]]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
ohe.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=ohe.get_feature_names_out(["Geography"]))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


Concatenating the encoded column with original dataframe

In [12]:
df=pd.concat([df.drop("Geography",axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


Saving the encoders and Scalers

In [10]:
with open('le_gender.pkl', 'wb') as file:
    pickle.dump(le_gender, file)

with open('ohe.pkl', 'wb') as file:
    pickle.dump(ohe, file)

dividing the dataset in to independent and dependent(X and Y)

In [13]:
X=df.iloc[:,:-1]
y=df['Exited']
X,y

(      CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
 0             619       0   42       2       0.00              1          1   
 1             608       0   41       1   83807.86              1          0   
 2             502       0   42       8  159660.80              3          1   
 3             699       0   39       1       0.00              2          0   
 4             850       0   43       2  125510.82              1          1   
 ...           ...     ...  ...     ...        ...            ...        ...   
 9995          771       1   39       5       0.00              2          1   
 9996          516       1   35      10   57369.61              1          1   
 9997          709       0   36       7       0.00              1          0   
 9998          772       1   42       3   75075.31              2          1   
 9999          792       0   28       4  130142.79              1          1   
 
       IsActiveMember  EstimatedSalary

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [14]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [16]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ..., -0.50857963,
         1.00150113, -0.57946723],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.50857963,
        -0.99850112,  1.72572313],
       [-0.96147213,  0.91324755, -1.41636539, ...,  1.96626042,
        -0.99850112, -0.57946723],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.96626042,
         1.00150113, -0.57946723],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.96626042,
         1.00150113, -0.57946723],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.50857963,
        -0.99850112,  1.72572313]])

In [17]:
y_train

9254    0
1561    0
1670    1
6087    1
6669    1
       ..
5734    0
5191    0
5390    1
860     1
7270    0
Name: Exited, Length: 8000, dtype: int64

ANN Implementation

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [19]:
X_train.shape

(8000, 12)

In [20]:
model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1,activation="sigmoid")
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

Setting up tensorboard

In [23]:
log_dir="logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback= TensorBoard(log_dir=log_dir,histogram_freq=1)

Setting up Early Stopping

In [24]:
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

Training the model

In [25]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 4s 7ms/step - loss: 0.1151 - accuracy: 0.9611 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 8.0143e-04 - val_accuracy: 1.0000
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 5.2239e-04 - accuracy: 1.0000 - val_loss: 3.4076e-04 - val_accuracy: 1.0000
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 2.5090e-04 - accuracy: 1.0000 - val_loss: 1.8629e-04 - val_accuracy: 1.0000
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 1.4606e-04 - accuracy: 1.0000 - val_loss: 1.1586e-04 - val_accuracy: 1.0000
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 9.4625e-05 - accuracy: 1.0000 - val_loss: 7.8328e-05 - val_accuracy: 1.0000
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 6.5577e-05 - accuracy: 1.0000

In [26]:
model.save("model.h5")

d:\MyLearning\annclassification\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Load the tensorboard extension

In [27]:
%load_ext tensorboard

In [30]:
%tensorboard --logdir logs/20240815-111455/

Reusing TensorBoard on port 6007 (pid 11256), started 0:00:49 ago. (Use '!kill 11256' to kill it.)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,Estimated Salary
0,600,France,Male,40,3,60000,2,1,1,50000


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

,Geography_France
0,1.0


In [50]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,Estimated Salary
0,600,France,1,40,3,60000,2,1,1,50000


,CreditScore,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,Estimated Salary,Geography_France,Geography_Germany,Geography_Spain
0,600.0,1.0,40.0,3.0,60000.0,2.0,1,1.0,50000.0,1.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Estimated Salary
- Geography_Spain
- NumofProducts
Feature names seen at fit time, yet now missing:
- EstimatedSalary
- Exited
- NumOfProducts
